In [40]:
import numpy as np
import random
from more_itertools import distinct_permutations
import time
from tabulate import tabulate

In [41]:
class fif3puzzle:
    def __init__(self):
        self.actions_set = ["up", "down", "left", "right"]
        self.gamma = 0.9
        self.num_cells = 16
        self.step = 0

        self.solved1 = [1, 2, 3, 4, self.num_cells] + [0] * 11
        self.solved2 = [-1] * 4 + [5, 6, 7, 8, self.num_cells] + [0] * 7
        self.solvedall = [-1] * 8 + [9, 10, 11, 12, 13, 14, 15, self.num_cells]
        
        self.phase1_states = self.generate_states([1, 2, 3, 4, self.num_cells], 0, 11)
        self.pol1 = self.pol_itr(self.phase1_states)
        
        self.phase2_states = self.generate_states([5, 6, 7, 8, self.num_cells], 4, 7)
        self.pol2 = self.pol_itr(self.phase2_states)
        
        self.phase3_states = self.generate_states([9, 10, 11, 12, 13, 14, 15, self.num_cells], 8, 0)
        self.pol3 = self.pol_itr(self.phase3_states)
    
    def display_state(self, state, step_cnt):
        if step_cnt>0 :
            print(f"Step: {step_cnt}")
        grid = [[str(x) if x != self.num_cells else "X" for x in state[i:i+4]] for i in range(0, 16, 4)]
        print(tabulate(grid, tablefmt="double_grid"))
        print("\n")
        time.sleep(1)
    
    def howmuchsolved(self, state):
        return (state[:4] == self.solved1[:4] or
                state[:8] == self.solved2[:8] or
                state == self.solvedall)
    
    def mask_state(self, state, row):
        masked_state = state.copy()
        
        if row == 1:
            return [x if x in [1, 2, 3, 4, self.num_cells] else 0 for x in masked_state]
        elif row == 2:
            return [-1 if x in [1, 2, 3, 4] else x if x in [5, 6, 7, 8, self.num_cells] else 0 for x in masked_state]
        elif row == 3:
            return [-1 if x in [1, 2, 3, 4, 5, 6, 7, 8] else x for x in masked_state]
        return masked_state
    
    def posact(self, state):
        valid_moves = []
        n = state.index(self.num_cells)
        row = (n//4)
        col = (n%4) 
        
        if row > 0:
            valid_moves.append("up")
        if row < 3:
            valid_moves.append("down")
        if col > 0:
            valid_moves.append("left")
        if col < 3:
            valid_moves.append("right")
        
        return valid_moves
    
    def mask_move(self, state, action):
        i = state.index(self.num_cells)
        row = (i//4)
        col = (i%4)
        
        if action == "up" and row > 0 and state[i - 4] != -1:
            state[i], state[i - 4] = state[i - 4], state[i]
        elif action == "down" and row < 3 and state[i + 4] != -1:
            state[i], state[i + 4] = state[i + 4], state[i]
        elif action == "left" and col > 0 and state[i - 1] != -1:
            state[i], state[i - 1] = state[i - 1], state[i]
        elif action == "right" and col < 3 and state[i + 1] != -1:
            state[i], state[i + 1] = state[i + 1], state[i]
        
        return state
    
    def move(self, state, action):
        return self.mask_move(state.copy(), action)
    
    def reward(self, state):
        return 1000 if self.howmuchsolved(state) else -1
    
    def generate_states(self, distinct_numbers, num_minus_ones, num_zeros):
        arr = distinct_numbers + [0] * num_zeros
        return [[-1] * num_minus_ones + list(perm) for perm in distinct_permutations(arr)]
    
    def pol_itr(self, states):
        V = {tuple(state): 0 for state in states}
        pol = {tuple(state): '' for state in states}
        stable = False
        while not stable:
            stable = self.pol_improv(V, pol, states)
            self.pol_eval(V, pol, states)
        return pol
    
    def pol_improv(self, V, pol, states):
        stable = True
        for state in states:
            state_tuple = tuple(state)
            if self.howmuchsolved(state):
                continue
            max_value, best_action = -np.inf, ''
            for action in self.posact(state):
                new_state = self.mask_move(state.copy(), action)
                action_value = self.reward(new_state) + self.gamma * V[tuple(new_state)]
                if action_value > max_value:
                    max_value, best_action = action_value, action
            if best_action != pol[state_tuple]:
                stable = False
            pol[state_tuple] = best_action
        return stable
    
    def pol_eval(self, V, pol, states):
        for state in states:
            state_tuple = tuple(state)
            if self.howmuchsolved(state):
                continue
            new_state = self.mask_move(state.copy(), pol[state_tuple])
            V[state_tuple] = self.reward(new_state) + self.gamma * V[tuple(new_state)]
    
    def apply_pol(self, state, pol, phase):
        actions = []
        masked_state = self.mask_state(state, phase)
        while True:
            action = pol.get(tuple(masked_state), '')
            if action not in self.actions_set:
                break
            actions.append(action)
            masked_state = self.move(masked_state, action)
        return actions
    
    def scramble_and_solve(self, num_moves=150,step_cnt=0):
        state = list(range(1, 16)) + [self.num_cells]
        for _ in range(num_moves):
            action = random.choice(self.posact(state))
            state = self.move(state, action)
    
        print("the cards we were dealt :")
        self.display_state(state,step_cnt)
        
        
        for phase, pol in enumerate([self.pol1, self.pol2, self.pol3], 1):
            actions = self.apply_pol(state, pol, phase)
            for action in actions:
                state = self.move(state, action)
                step_cnt+=1
                self.display_state(state,step_cnt)
        
        return state

solver = fif3puzzle()

In [42]:

final_state = solver.scramble_and_solve()
print("aaaand we're done !!!!!!!")

the cards we were dealt :
╔════╦════╦════╦════╗
║  7 ║  5 ║ X  ║  2 ║
╠════╬════╬════╬════╣
║  9 ║  6 ║ 1  ║  4 ║
╠════╬════╬════╬════╣
║ 13 ║ 10 ║ 12 ║ 11 ║
╠════╬════╬════╬════╣
║ 14 ║  3 ║ 15 ║  8 ║
╚════╩════╩════╩════╝


Step: 1
╔════╦════╦════╦════╗
║  7 ║  5 ║ 1  ║  2 ║
╠════╬════╬════╬════╣
║  9 ║  6 ║ X  ║  4 ║
╠════╬════╬════╬════╣
║ 13 ║ 10 ║ 12 ║ 11 ║
╠════╬════╬════╬════╣
║ 14 ║  3 ║ 15 ║  8 ║
╚════╩════╩════╩════╝


Step: 2
╔════╦════╦════╦════╗
║  7 ║  5 ║ 1  ║  2 ║
╠════╬════╬════╬════╣
║  9 ║  6 ║ 12 ║  4 ║
╠════╬════╬════╬════╣
║ 13 ║ 10 ║ X  ║ 11 ║
╠════╬════╬════╬════╣
║ 14 ║  3 ║ 15 ║  8 ║
╚════╩════╩════╩════╝


Step: 3
╔════╦═══╦════╦════╗
║  7 ║ 5 ║  1 ║  2 ║
╠════╬═══╬════╬════╣
║  9 ║ 6 ║ 12 ║  4 ║
╠════╬═══╬════╬════╣
║ 13 ║ X ║ 10 ║ 11 ║
╠════╬═══╬════╬════╣
║ 14 ║ 3 ║ 15 ║  8 ║
╚════╩═══╩════╩════╝


Step: 4
╔════╦═══╦════╦════╗
║  7 ║ 5 ║  1 ║  2 ║
╠════╬═══╬════╬════╣
║  9 ║ 6 ║ 12 ║  4 ║
╠════╬═══╬════╬════╣
║ 13 ║ 3 ║ 10 ║ 11 ║
╠════╬═══╬════╬════╣
║ 14